# Data Preparation
The purpose of this notebook is to **preprocess** and **explore** the credit
approval dataset. At the beginning of each section, we will summarize the steps
taken to preprocess the dataset and then explore it.

In [1]:
# Load Dependencies
import numpy as np
import pandas as pd
import torch
import sklearn
import matplotlib.pyplot as plt

In [2]:
# Load Raw Dataset
filepath = "/Users/drewgjerstad/repos/credit-approval-prediction/data/crx.data"

credit_data = pd.read_table(filepath_or_buffer=filepath,
                            delimiter=",", na_values=["?"],
                            names=["A1", "A2", "A3", "A4", "A5", "A6", "A7",
                                   "A8", "A9", "A10", "A11", "A12", "A13",
                                   "A14", "A15", "A16"])
credit_data.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+


## Data Preprocessing

## Data Exploration